# Huspriser data cleaner

In [121]:
# importer pakker
import numpy as np
import pandas as pd

In [122]:
# importer data
data = pd.read_csv('HousingPrices_original.csv', sep=';')

/var/folders/cv/4hkgm9gn1dv2hp0k2jlt7qrm0000gn/T/ipykernel_27701/1922117324.py:2: DtypeWarning: Columns (31,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('HousingPrices_original.csv', sep=';')


In [123]:
keep_and_rename = {
     'MI_OBJ_OIS_POSTAL_CODE': 'Postnummer',
     'MI_OBJ_OIS_HOUSE_NUMBER': 'Husnummer',
     'MI_OBJ_OIS_SIZE_OF_HOUSE': 'Kvadratmeter',
     'MI_OBJ_OIS_SIZE_OF_PLOT': 'Grundareal',
     'MI_OBJ_OIS_CONSTRUCTION_YEAR': 'Byggeår',
     'SCHOOL_DISTANCE_1': 'Afstand_skole',
     'SUPERMARKET_DISTANCE_1': 'Afstand_supermarked',
     'KOEBESUM_BELOEB': 'Salgspris',
     'MI_OBJ_OIS_FLOOR_NUMBER': 'Etage'
}


In [124]:
existing = {old: new for old, new in keep_and_rename.items() if old in data.columns}
missing  = [old for old in keep_and_rename if old not in data.columns]

subset = data[list(existing.keys())].rename(columns=existing)
print(f'datapoints: {len(subset)}')

datapoints: 49289


In [125]:
# Create a new column for type
subset['Type'] = pd.Series([pd.NA] * len(subset), dtype='Int64')


mask = subset['Etage'].notna() & subset['Etage'].astype(str).str.strip().ne('')
subset['Type'] = np.where(mask, 1, 2)  

print(subset['Type'])

0        2
1        2
2        2
3        1
4        1
        ..
49284    2
49285    2
49286    2
49287    1
49288    1
Name: Type, Length: 49289, dtype: int64


In [126]:
# Fjern alle over 10 mio

subset = subset[subset['Salgspris'].le(10_000_000)]
print(f'datapoints: {len(subset)}')
subset.isna().sum().sort_values(ascending=True)


datapoints: 48709


Kvadratmeter             0
Salgspris                0
Type                     0
Husnummer                4
Grundareal               4
Etage                    4
Postnummer              18
Afstand_skole           35
Afstand_supermarked     35
Byggeår                992
dtype: int64

In [127]:
# Fjerner alle rækker med NaN

data_no_na = subset.dropna()  
print(f'datapoints: {len(data_no_na)}')                
data_no_na.to_csv('HousingPrices_selected.csv', index=False)  
data_no_na.isna().sum().sort_values(ascending=True)  

datapoints: 47708


Postnummer             0
Husnummer              0
Kvadratmeter           0
Grundareal             0
Byggeår                0
Afstand_skole          0
Afstand_supermarked    0
Salgspris              0
Etage                  0
Type                   0
dtype: int64

#